# Local DEM Mount Testing Notebook

This notebook just exists to test if PCM can mount in the local DEM archive directory in the job image this is running in.

In [ ]:
track = 1 # int
frame = 5 # int

# hysds specifications
_time_limit = 172800
_soft_time_limit = 172800
_disk_usage = '200GB'
_submission_type = 'iteration'
_label = 'Test DEM Archive PGE'

In [ ]:
import os
from pathlib import Path

# Define the relative paths to the folders and files we want to test
folders_to_list = [
    Path("."),                  # Root
    Path("dem_archive"),
    Path("dem_archive/track_1")
]

files_to_open = [
    Path("dem_archive/track_1/dem_1_5_0.vrt"),
    Path("dem_archive/track_1/dem_1_5_0.tiff")
]

# Define the two base prefixes: HOME and Current Working Directory (CWD)
prefixes = {
    "HOME": Path.home(),
    "CWD": Path.cwd()
}

print(f"{'TESTING DIRECTORY LISTINGS':=^60}")

for name, base_path in prefixes.items():
    print(f"\n--- Using Prefix: {name} ({base_path}) ---")
    
    for folder in folders_to_list:
        target_path = base_path / folder
        print(f"\n📁 Checking: {target_path}")
        
        try:
            if target_path.exists() and target_path.is_dir():
                # List contents (folders first, then files)
                contents = sorted(list(target_path.iterdir()), key=lambda x: (not x.is_dir(), x.name))
                for item in contents:
                    type_label = "[DIR] " if item.is_dir() else "[FILE]"
                    print(f"  {type_label} {item.name}")
                if not contents:
                    print("  (Empty directory)")
            else:
                print(f"  ❌ Directory not found or is not a directory.")
        except PermissionError:
            print(f"  ❌ Permission denied.")
        except Exception as e:
            print(f"  ❌ Error: {e}")

print(f"\n\n{'TESTING FILE ACCESS':=^60}")

for name, base_path in prefixes.items():
    print(f"\n--- Using Prefix: {name} ---")
    
    for file_rel in files_to_open:
        target_file = base_path / file_rel
        try:
            # Attempt to open the file in binary read mode
            with open(target_file, 'rb') as f:
                print(f"✅ Success: Opened {target_file.name}")
        except FileNotFoundError:
            print(f"❌ Missing: {target_file}")
        except PermissionError:
            print(f"❌ Permission Denied: {target_file}")
        except Exception as e:
            print(f"❌ Error ({e}): {target_file}")